# Astrogator Object Model Walkthrough in Python#

## Connect to STK ##

In [148]:
# Import Libraries
import comtypes
from comtypes.client import CreateObject
from comtypes.gen import STKObjects
from comtypes.gen import STKUtil
from comtypes.gen import AgStkGatorLib

In [122]:
# Create a new instance of STK, grab the root, create a new scenario
app = CreateObject('STK12.Application')
app.Visible = 1
app.UserControl = 1
root = app.Personality2
root.NewScenario('HohmannTransferPython');

In [123]:
# Grab scenario handles
sc = root.CurrentScenario
sc2 = sc.QueryInterface(STKObjects.IAgScenario)

## Create an Astrogator Satellite ##

In [124]:
# Create a new satellite
sat = sc.Children.New(STKObjects.eSatellite,'AstrogatorSat')

In [125]:
# Get a handle to the IAgSatellite interface
sat2 = sat.QueryInterface(STKObjects.IAgSatellite)
# Set the propagator to ePropagatorAstrogator
sat2.SetPropagatorType(STKObjects.ePropagatorAstrogator)
# Cast the propagator to an IAgVADriverMCS
ASTG = sat2.Propagator.QueryInterface(AgStkGatorLib.IAgVADriverMCS)


In [126]:
# Get a handle to the MCS
MCS = ASTG.MainSequence
type(MCS)

comtypes.POINTER(IAgVAMCSSegmentCollection)

## Build the MCS ## 

In [127]:
ts1a = MCS.Insert(AgStkGatorLib.eVASegmentTypeTargetSequence,'TargetApo','-')
type(ts1a)

comtypes.POINTER(IAgVAMCSSegment)

In [128]:
ts1b = ts1a.QueryInterface(AgStkGatorLib.IAgVAMCSTargetSequence)

In [129]:
dV1a = ts1b.Segments.Insert(2,'dV1','-')
type(dV1a)

comtypes.POINTER(IAgVAMCSSegment)

In [130]:
dV1b = dV1a.QueryInterface(AgStkGatorLib.IAgVAMCSManeuver)

In [131]:
dV1b.Maneuver.SetAttitudeControlType(AgStkGatorLib.eVAAttitudeControlThrustVector)
type(dV1b.Maneuver.AttitudeControl)

comtypes.POINTER(IAgVAAttitudeControl)

In [132]:
attCon = dV1b.Maneuver.AttitudeControl.QueryInterface(AgStkGatorLib.IAgVAAttitudeControlImpulsiveThrustVector)

In [133]:
attCon.DeltaVVector.AssignCartesian(1,0,0);

In [134]:
prop2a = ts1b.Segments.Insert(AgStkGatorLib.eVASegmentTypePropagate,'PropToApo','-')
prop2b = prop2a.QueryInterface(AgStkGatorLib.IAgVAMCSPropagate)

In [135]:
prop2b.StoppingConditions.Add('Apoapsis')

<POINTER(IAgVAStoppingConditionElement) ptr=0x1b74ddc41a8 at 1b74f1db348>

In [138]:
ts2a = MCS.Insert(AgStkGatorLib.eVASegmentTypeTargetSequence,'TargetCircularization','-')
ts2b = ts2a.QueryInterface(AgStkGatorLib.IAgVAMCSTargetSequence)

In [139]:
dV2a = ts2b.Segments.InsertCopy(dV1a,'-')
dV2b = dV2a.QueryInterface(AgStkGatorLib.IAgVAMCSManeuver)
dV2a.Properties

In [140]:
prop3a = ts2b.Segments.Insert(AgStkGatorLib.eVASegmentTypePropagate,'Prop1Day','-')
prop3b = prop3a.QueryInterface(AgStkGatorLib.IAgVAMCSPropagate)

In [141]:
scDura = prop3b.StoppingConditions.Item(0)
scDurb = scDura.Properties.QueryInterface(AgStkGatorLib.IAgVAStoppingCondition)
scDurb.Trip = 86400

## Configure the Target Sequences ## 

In [158]:
dV1b.EnableControlParameter(AgStkGatorLib.eVAControlManeuverImpulsiveCartesianX)
dV2b.EnableControlParameter(AgStkGatorLib.eVAControlManeuverImpulsiveCartesianX)
dV2b.EnableControlParameter(AgStkGatorLib.eVAControlManeuverImpulsiveCartesianY);

In [169]:
# Add results
dV1a.Results.Add('Spherical Elems\\R Mag')
dV2a.Results.Add('')

<POINTER(IAgComponentInfo) ptr=0x1b745de8ad8 at 1b75fa3e0c8>

In [144]:
# Set the Target Profiles Action to Run active profiles 
ts1b.Action = AgStkGatorLib.eVATargetSeqActionRunActiveProfiles
ts2b.Action = AgStkGatorLib.eVATargetSeqActionRunActiveProfiles

## Run the MCS ##

In [156]:
dV2b = dV2a.QueryInterface(AgStkGatorLib.IAgVAMCSManeuver)

In [161]:
type(dV1a.Results)

comtypes.POINTER(IAgVACalcObjectCollection)